个人经常在选择损失函数的时候犯迷糊，所以这篇文章是记录一下pytorch常见损失函数的数学表达式及其实现的

In [1]:
import torch
import torch.nn as nn

# 构造数据，假设batch_size=5，标签是10个类别
y_hat = torch.zeros(5, 10)
y = torch.ones(5, 10)
y_i = torch.tensor([1, 2, 3, 1, 2])

## L1Loss损失函数
计算MAE，即差值的绝对值，就是一个L1范数所以也叫做L1Loss，假设$\hat{y}$是预测值，$y$是真实的标签值，那么这个损失值就用如下的数学表达式表示：
$$
loss\_value = |\hat{y} - y|
$$
在pytorch中，该损失函数是一个类，需要初始化后才能使用，有三个初始化参数：
- ~~size_average~~, 
- ~~reduce,~~ 
- reduction，'sum' or 'mean'，'sum'表示对所有损失值进行求和，'mean'表示求当前batch总体的损失值。默认是取值'mean'
三个参数其实描述的是同一个东西，但前两个参数是过时的（[见文档](https://pytorch.org/docs/stable/nn.html#l1loss)），所以我们只需要使用**reduction**参数就好了,**后续的损失函数都是如此设计的，就不会再陈述了**

In [2]:
L1_mean = nn.L1Loss(reduction='mean')
loss_mean = L1_mean(y_hat, y)
print(loss_mean)

L1_sum = nn.L1Loss(reduction='sum')
loss_sum = L1_sum(y_hat, y)
print(loss_sum)

tensor(1.)
tensor(50.)


## MSELoss损失函数
计算均方误差，数学表达式如下：
$$
loss\_value = (y - \hat{y})^2
$$

In [3]:
mse_mean = nn.MSELoss(reduction='mean')
mse_sum = nn.MSELoss(reduction='sum')
loss_mean = mse_mean(y, y_hat)
loss_sum = mse_sum(y, y_hat)
print(loss_mean)
print(loss_sum)

tensor(1.)
tensor(50.)


## CrossEntropyLoss
交叉熵函数，**适用于多分类问题**，数学表达式如下：
$$
loss\_value(x, class) = -log(\frac{e^{x[class]}}{\sum_{j}{e^{x[j]}}}) = -x[class] + log(\sum_{j} x[j])
$$
其中，x是一个1D tensor，class是下标（或者标签）。

对于类的初始化，除了考虑**reduction**这个参数，还有如下几个参数：
- weight, 一个与类别数量长度相同的tensor，用了这个参数的话，我们的损失函数就变为了：$loss\_value(x, class) = weight[class](-x[class] + log(\sum_{j} x[j]))$
- ignore_index ,范围是[0, C-1]之间的整数，C表示类别的数量。这个参数是一个整数表明某个类别不需要参与损失函数的计算，相当于少了某一个类。


In [14]:
cel_sum = nn.CrossEntropyLoss(reduction='sum')
cel_mean = nn.CrossEntropyLoss(reduction='mean')
# 第一个参数是形如(N, C)的tensor，第二个参数是（N，）的tensor，每个值是第一个参数C的某个下标
loss_sum = cel_sum(y_hat, y_i)
loss_mean = cel_mean(y_hat, y_i)
print(loss_sum)
print(loss_mean)

print('weight...')
weight = torch.tensor([0.1, 0.2, 0.3, 0.2, 0.2, 0, 0, 0, 0, 0])
cel_sum_w = nn.CrossEntropyLoss(weight=weight, reduction='sum')
cel_mean_w = nn.CrossEntropyLoss(weight=weight, reduction='mean')
# 第一个参数是形如(N, C)的tensor，第二个参数是（N，）的tensor，每个值是第一个参数C的某个下标
loss_sum = cel_sum_w(y_hat, y_i)
loss_mean = cel_mean_w(y_hat, y_i)
print(loss_sum)
print(loss_mean)

print('ignore index...')
cel_sum = nn.CrossEntropyLoss(ignore_index=1,reduction='sum')
cel_mean = nn.CrossEntropyLoss(ignore_index=1, reduction='mean')
# 第一个参数是形如(N, C)的tensor，第二个参数是（N，）的tensor，每个值是第一个参数C的某个下标
loss_sum = cel_sum(y_hat, y_i)
loss_mean = cel_mean(y_hat, y_i)
print(loss_sum)
print(loss_mean)

tensor(11.5129)
tensor(2.3026)
weight...
tensor(2.7631)
tensor(2.3026)
ignore index...
tensor(6.9078)
tensor(2.3026)
